In [4]:
import pandas as pd

# dados que vou usar para a agregação
ibge = pd.read_csv('gs://trabalho-final-pdm/silver/ibge_goiania.csv', storage_options={"token": "google_default"})

# 2016
tse = pd.read_csv('gs://trabalho-final-pdm/silver/tse_bairros_2024.csv', storage_options={"token": "google_default"})


zonas = pd.read_csv('gs://trabalho-final-pdm/bronze/zonas_goiania', storage_options={"token": "google_default"})

/tmp/ipykernel_29938/1660236506.py:7: DtypeWarning: Columns (12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  tse = pd.read_csv('gs://trabalho-final-pdm/silver/tse_bairros_2024.csv', storage_options={"token": "google_default"})


In [5]:
ibge.head()

,Nome_do_subdistrito,Nome_do_bairro,Situacao_setor,V001,V002,V005,V007,V009,V011
0,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,276.0,642.0,"2432,47","2543,03","1588,1","2084,38"
1,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,225.0,491.0,"2198,14","2389,29","1512,69","2008,52"
2,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,250.0,584.0,"1787,44","1885,48","1233,05","1513,68"
3,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,217.0,363.0,"2209,32","2282,96","1730,26","2051,34"
4,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,214.0,571.0,"3044,29","3257,39","1871,55","2607,5"


In [6]:
tse.head()

,ANO_ELEICAO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_TIPO_ELEICAO,NR_TURNO,DS_CARGO,NM_VOTAVEL,NR_VOTAVEL,QT_VOTOS,NM_LOCAL_VOTACAO,DS_LOCAL_VOTACAO_ENDERECO,LOCAL DE VOTACAO,ENDEREÇO,BAIRRO
0,2024,NEROPOLIS,54,22,2,1,VEREADOR,VALMI BARBOSA DOS SANTOS JUNIOR,11111,6,COLEGIO ESTADUAL DR NEGREIROS,RUA NARCEU DE ALMEIDA N. 220,NaN,NaN,NaN
1,2024,SANTA HELENA DE GOIAS,66,89,2,1,VEREADOR,ANISIO VICENTE FERREIRA FILHO,27000,1,CENTRO DE ENSINO EM PERIODO INTEGRAL JOSE SALV...,"RUA JACINTO FERREIRA DE SOUZA, 950",NaN,NaN,NaN
2,2024,GOIANESIA,74,244,2,1,VEREADOR,ALINE FERNANDES,22522,3,ESCOLA MUNICIPAL LAURO DA PENHA,"RUA ANDORINHA, N 466",NaN,NaN,NaN
3,2024,GOIANESIA,74,125,2,1,VEREADOR,ISMAEL FRANCISCO DE ASSIS,45789,1,ESCOLA MUNICIPAL PROFESSOR JUDITH LEITE,RUA 20 N 597,NaN,NaN,NaN
4,2024,GOIANESIA,74,244,2,1,VEREADOR,ISMAEL FRANCISCO DE ASSIS,45789,4,ESCOLA MUNICIPAL LAURO DA PENHA,"RUA ANDORINHA, N 466",NaN,NaN,NaN


In [7]:
zonas.head()

,ZONA,LOCAL DE VOTACAO,ENDEREÇO,BAIRRO,total de seções,total de eleitores
0,1,Colegio Agostiniano Nossa Senhora de Fatima,"Av K, 108 - Unidade I",Setor Aeroporto,19,7.291
1,147,Colegio Desafio,Av Mangalo Qd 55 Lt 06,Setor Morada do Sol,18,6.944
2,136,Colegio Estadual Jardim Vila Boa,Rua Paola Ney Qd. 59,Jardim Vila Boa,20,6.875
3,127,Escola Sesi Jardim Planalto,Praca Itapua Qd 30 150,Jardim Planalto,20,6.524
4,127,Colegio Estadual Polivalente Profº Goiany Prates,Rua C-68 Qd 115 336,Vilas dos Alpes,17,6.389


In [8]:
tse.shape

(1320312, 15)

In [9]:
# ultimas transformações em IBGE para a agregação
# Vamos ter que entender essa tabela como uma amostra, porque eu filtrei teoricamente pela cidade, então, 
# se o censo passar por todas as casas, no minimo deveria ter 1 milhão de linhas esse dataset

# Converter as colunas numéricas para o tipo float
ibge["V005"] = ibge["V005"].str.replace(",", ".").astype(float)
ibge["V009"] = ibge["V009"].str.replace(",", ".").astype(float)

# De salarios deixei apenas o que tenho certeza
# Acho que esses dados representam o setor cencitário, mas queremos so valores por setor

ibge = ibge[['Nome_do_subdistrito', 'Situacao_setor', 'V005', 'V009']]

# Converter as colunas numéricas para o tipo float
ibge["V005"] = pd.to_numeric(ibge["V005"], errors="coerce")
ibge["V009"] = pd.to_numeric(ibge["V009"], errors="coerce")


# Agregar os dados por Nome_do_subdistrito
ibge_agrupado = ibge.groupby("Nome_do_subdistrito").agg({
    "Situacao_setor": lambda x: x.mode()[0],  # Pega o valor mais comum
    "V005": "mean",  # Média dos valores
    "V009": "mean",  # Média dos valores
}).reset_index()

# Ver o resultado
ibge_agrupado

,Nome_do_subdistrito,Situacao_setor,V005,V009
0,U.T.P. AEROPORTO,1,3679.232000,2220.962000
1,U.T.P. AEROPORTO INTERNACIONAL SANTA GENOVEVA,1,3435.710000,1542.860000
2,U.T.P. AEROVIARIOS,1,1243.974737,806.283158
3,U.T.P. ALTO DA GLORIA E REDENCAO,1,3067.470400,2045.876400
4,U.T.P. AUTODROMO OU PARQUE LOZANDES,1,8360.048000,4542.501000
...,...,...,...,...
59,U.T.P. VILA PEDROSO,1,880.640800,577.509600
60,U.T.P. VILA REGINA/PARQUE INDUSTRIAL PAULISTA/...,1,1199.879286,750.282143
61,U.T.P. VILA RIZZO,1,874.826000,597.820667
62,U.T.P. ZONA RURAL,1,860.635952,536.335714


In [7]:
!pip install scikit-learn

In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Função para imputar bairros usando TF-IDF
def imputar_bairros(tse_texts, zonas_texts, zonas_df, threshold=0.7):
    """
    Parâmetros:
    tse_texts: Lista de textos da tabela tse (NM_LOCAL_VOTACAO)
    zonas_texts: Lista de textos da tabela zonas (LOCAL DE VOTACAO)
    zonas_df: DataFrame com dados da tabela zonas
    threshold: Valor mínimo de similaridade para imputação
    """
    vectorizer = TfidfVectorizer()
    vectorizer.fit(zonas_texts)

    # Vetorizar os textos
    tse_vetorizado = vectorizer.transform(tse_texts)
    zonas_vetorizado = vectorizer.transform(zonas_texts)

    # Calcular similaridade
    similaridades = cosine_similarity(tse_vetorizado, zonas_vetorizado)

    # Criar lista com bairros imputados
    bairros_imputados = []
    for i, similaridades_linha in enumerate(similaridades):
        max_similaridade = similaridades_linha.max()
        if max_similaridade >= threshold:
            idx = similaridades_linha.argmax()
            bairro = zonas_df.iloc[idx]["BAIRRO"]
        else:
            bairro = None  # Sem similaridade suficiente
        bairros_imputados.append(bairro)

    return bairros_imputados

# Preparar os textos
tse_texts = tse["NM_LOCAL_VOTACAO"].fillna("").tolist()
zonas_texts = zonas["LOCAL DE VOTACAO"].fillna("").tolist()

# Imputar os bairros
bairros_imputados = imputar_bairros(tse_texts, zonas_texts, zonas, threshold=0.7)

# Adicionar a nova coluna 'bairros_imputados' no DataFrame tse
tse["bairros_imputados"] = bairros_imputados

# Exibir o resultado final
tse


,ANO_ELEICAO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_TIPO_ELEICAO,NR_TURNO,DS_CARGO,NM_VOTAVEL,NR_VOTAVEL,QT_VOTOS,NM_LOCAL_VOTACAO,DS_LOCAL_VOTACAO_ENDERECO,LOCAL DE VOTACAO,ENDEREÇO,BAIRRO,bairros_imputados
0,2024,NEROPOLIS,54,22,2,1,VEREADOR,VALMI BARBOSA DOS SANTOS JUNIOR,11111,6,COLEGIO ESTADUAL DR NEGREIROS,RUA NARCEU DE ALMEIDA N. 220,NaN,NaN,NaN,None
1,2024,SANTA HELENA DE GOIAS,66,89,2,1,VEREADOR,ANISIO VICENTE FERREIRA FILHO,27000,1,CENTRO DE ENSINO EM PERIODO INTEGRAL JOSE SALV...,"RUA JACINTO FERREIRA DE SOUZA, 950",NaN,NaN,NaN,None
2,2024,GOIANESIA,74,244,2,1,VEREADOR,ALINE FERNANDES,22522,3,ESCOLA MUNICIPAL LAURO DA PENHA,"RUA ANDORINHA, N 466",NaN,NaN,NaN,None
3,2024,GOIANESIA,74,125,2,1,VEREADOR,ISMAEL FRANCISCO DE ASSIS,45789,1,ESCOLA MUNICIPAL PROFESSOR JUDITH LEITE,RUA 20 N 597,NaN,NaN,NaN,None
4,2024,GOIANESIA,74,244,2,1,VEREADOR,ISMAEL FRANCISCO DE ASSIS,45789,4,ESCOLA MUNICIPAL LAURO DA PENHA,"RUA ANDORINHA, N 466",NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320307,2024,IPAMERI,14,35,2,1,VEREADOR,CLEBERSON ANTONIO LUIZ,55123,3,ESCOLA ESTADUAL MICHELE SANTINONI,AV. TEODORO SAMPAIO NR 23,NaN,NaN,NaN,None
1320308,2024,GOIANIA,147,31,2,1,VEREADOR,MARCOS ANTONIO DA SILVA,25456,1,ESCOLA MUNICIPAL PEDRO GOMES DE MENEZES,RUA MANOEL SILVA QD 08 LT 01 VILA REGINA,"['COLEGIO DESAFIO', 'COLEGIO ESTADUAL PROFESSO...","['AV MANGALO QD 55 LT 06', 'AV. FERNAO DIAS PA...","['SETOR MORADA DO SOL', 'CAPUAVA', 'VILA MULTI...",Vila Regina
1320309,2024,GOIANIA,2,367,2,1,VEREADOR,MARCOS ANTONIO DA SILVA,25456,2,CENTRO DE ENSINO EM PERIODO INTEGRAL AECIO OLI...,RUA CARIJOS ESQ C/ RUA DO PRATA S/N,['CENTRO DE ENSINO EM PERIODO INTEGRAL AECIO O...,"['RUA CARIJOS COM RUA DO PRATA', 'AV SOL NASCE...","['SETOR URIAS MAGALHAES', 'JARDIM NOVA ESPERAN...",Setor Urias Magalhães
1320310,2024,GOIANIA,136,362,2,1,VEREADOR,MARCOS ANTONIO DA SILVA,25456,1,ESCOLA GOVERNADOR JOAQUIM SOBROSA - LIONS,R.ESTRADA DUQUE DE CAXIAS QD 3 LT 48/49,"['COLEGIO ESTADUAL JARDIM VILA BOA', 'COLEGIO ...","['RUA PAOLA NEY QD. 59', 'RUA B-16 101', 'RUA ...","['JARDIM VILA BOA', 'VILA NOVO HORIZONTE', 'SE...",Setor Cristina


In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Função para encontrar os melhores matches com um threshold
def match_tfidf(tse_texts, ibge_texts, ibge_df, threshold=0.5):
    """
    Parâmetros:
    tse_texts: Lista de textos da tabela tse
    ibge_texts: Lista de textos da tabela ibge
    ibge_df: DataFrame com dados do IBGE
    threshold: Limite mínimo de similaridade para considerar um match
    """
    vectorizer = TfidfVectorizer()
    vectorizer.fit(ibge_texts)

    # Vetorizar os textos
    tse_vetorizado = vectorizer.transform(tse_texts)
    ibge_vetorizado = vectorizer.transform(ibge_texts)

    # Calcular a similaridade de coseno
    similaridades = cosine_similarity(tse_vetorizado, ibge_vetorizado)

    # Encontrar o índice com maior similaridade
    melhores_matches = []
    for i, similaridades_linha in enumerate(similaridades):
        max_similaridade = similaridades_linha.max()
        if max_similaridade >= threshold:
            idx = similaridades_linha.argmax()
            match = ibge_df.iloc[idx].to_dict()
            match['similaridade'] = max_similaridade  # Adicionar a similaridade
        else:
            # Se a similaridade for menor que o threshold, criar linha vazia
            match = {col: None for col in ibge_df.columns}
            match['similaridade'] = max_similaridade
        melhores_matches.append(match)
    
    return melhores_matches

# Criar nova coluna combinada em tse (DS_LOCAL_VOTACAO_ENDERECO + bairros_imputados)
tse["combinado"] = tse["DS_LOCAL_VOTACAO_ENDERECO"].fillna("") + " " + tse["bairros_imputados"].fillna("")

# Criar nova coluna combinada em ibge (Nome_do_subdistrito, ou ajustar conforme necessário)
ibge["combinado"] = ibge["Nome_do_subdistrito"].fillna("")

# Preparar os textos para TF-IDF
tse_texts = tse["combinado"].tolist()
ibge_texts = ibge["combinado"].tolist()

# Definir o threshold mínimo de similaridade
threshold_similaridade = 0.5

# Realizar os matches com threshold
matches = match_tfidf(tse_texts, ibge_texts, ibge, threshold=threshold_similaridade)

# Combinar os DataFrames
tse_matches = pd.DataFrame(matches)
tse_unificado = pd.concat([tse.reset_index(drop=True), tse_matches.reset_index(drop=True)], axis=1)

# Exibir o resultado final
tse_unificado


,ANO_ELEICAO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_TIPO_ELEICAO,NR_TURNO,DS_CARGO,NM_VOTAVEL,NR_VOTAVEL,QT_VOTOS,...,ENDEREÇO,BAIRRO,bairros_imputados,combinado,Nome_do_subdistrito,Situacao_setor,V005,V009,combinado,similaridade
0,2024,NEROPOLIS,54,22,2,1,VEREADOR,VALMI BARBOSA DOS SANTOS JUNIOR,11111,6,...,NaN,NaN,None,RUA NARCEU DE ALMEIDA N. 220,None,NaN,NaN,NaN,None,0.487514
1,2024,SANTA HELENA DE GOIAS,66,89,2,1,VEREADOR,ANISIO VICENTE FERREIRA FILHO,27000,1,...,NaN,NaN,None,"RUA JACINTO FERREIRA DE SOUZA, 950",None,NaN,NaN,NaN,None,0.487514
2,2024,GOIANESIA,74,244,2,1,VEREADOR,ALINE FERNANDES,22522,3,...,NaN,NaN,None,"RUA ANDORINHA, N 466",None,NaN,NaN,NaN,None,0.000000
3,2024,GOIANESIA,74,125,2,1,VEREADOR,ISMAEL FRANCISCO DE ASSIS,45789,1,...,NaN,NaN,None,RUA 20 N 597,None,NaN,NaN,NaN,None,0.000000
4,2024,GOIANESIA,74,244,2,1,VEREADOR,ISMAEL FRANCISCO DE ASSIS,45789,4,...,NaN,NaN,None,"RUA ANDORINHA, N 466",None,NaN,NaN,NaN,None,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320307,2024,IPAMERI,14,35,2,1,VEREADOR,CLEBERSON ANTONIO LUIZ,55123,3,...,NaN,NaN,None,AV. TEODORO SAMPAIO NR 23,None,NaN,NaN,NaN,None,0.000000
1320308,2024,GOIANIA,147,31,2,1,VEREADOR,MARCOS ANTONIO DA SILVA,25456,1,...,"['AV MANGALO QD 55 LT 06', 'AV. FERNAO DIAS PA...","['SETOR MORADA DO SOL', 'CAPUAVA', 'VILA MULTI...",Vila Regina,RUA MANOEL SILVA QD 08 LT 01 VILA REGINA Vila...,U.T.P. VILA REGINA/PARQUE INDUSTRIAL PAULISTA/...,1.0,806.89,528.78,U.T.P. VILA REGINA/PARQUE INDUSTRIAL PAULISTA/...,0.514103
1320309,2024,GOIANIA,2,367,2,1,VEREADOR,MARCOS ANTONIO DA SILVA,25456,2,...,"['RUA CARIJOS COM RUA DO PRATA', 'AV SOL NASCE...","['SETOR URIAS MAGALHAES', 'JARDIM NOVA ESPERAN...",Setor Urias Magalhães,RUA CARIJOS ESQ C/ RUA DO PRATA S/N Setor Uria...,None,NaN,NaN,NaN,None,0.408829
1320310,2024,GOIANIA,136,362,2,1,VEREADOR,MARCOS ANTONIO DA SILVA,25456,1,...,"['RUA PAOLA NEY QD. 59', 'RUA B-16 101', 'RUA ...","['JARDIM VILA BOA', 'VILA NOVO HORIZONTE', 'SE...",Setor Cristina,R.ESTRADA DUQUE DE CAXIAS QD 3 LT 48/49 Setor ...,None,NaN,NaN,NaN,None,0.487514


In [12]:
tse_unificado.isnull().sum()

ANO_ELEICAO                       0
NM_MUNICIPIO                      0
NR_ZONA                           0
NR_SECAO                          0
CD_TIPO_ELEICAO                   0
NR_TURNO                          0
DS_CARGO                          0
NM_VOTAVEL                        0
NR_VOTAVEL                        0
QT_VOTOS                          0
NM_LOCAL_VOTACAO                  0
DS_LOCAL_VOTACAO_ENDERECO         0
LOCAL DE VOTACAO             956872
ENDEREÇO                     956872
BAIRRO                       956872
bairros_imputados            899269
combinado                         0
Nome_do_subdistrito          849676
Situacao_setor               849676
V005                         849676
V009                         849676
combinado                    849676
similaridade                      0
dtype: int64

In [13]:
tse_unificado.shape

(1320312, 23)

In [14]:
# seleção das features boas

tse_unificado = tse_unificado[['ANO_ELEICAO', 'NM_MUNICIPIO', 'NR_ZONA','NR_SECAO', 'CD_TIPO_ELEICAO', 'NR_TURNO', 'DS_CARGO', 'NM_VOTAVEL', 'NR_VOTAVEL', 'QT_VOTOS', 'NM_LOCAL_VOTACAO', 'Nome_do_subdistrito', 'Situacao_setor', 'V005', 'V009']]


In [15]:
# Salvar DataFrame diretamente no GCS
gcs_path = "gs://trabalho-final-pdm/silver/agregados_2024.csv"

# Escrever para o Cloud Storage usando storage_options
tse_unificado.to_csv(gcs_path, index=False, storage_options={"token": "google_default"})
